In [1]:
%run ./MovieLens_SequentialRecommendation.ipynb

100%|██████████| 99057/99057 [00:04<00:00, 21264.83it/s]


In [2]:
# 设置超参数
k = 20
d = 20
lr = 0.01
alpha_u = alpha_v = alpha_p = alpha_q = 0.01
epochs = 100

In [3]:
# 初始化模型参数
U = np.random.rand(user_num + 1, d)
V = np.random.rand(item_num + 1, d)
P = np.random.rand(item_num + 1, d)
Q = np.random.rand(item_num + 1, d)
U = (U - 0.5) * 0.01
V = (V - 0.5) * 0.01
P = (P - 0.5) * 0.01
Q = (Q - 0.5) * 0.01

In [4]:
# 计算用户u对session中第t个物品第偏好
def predict_rule(u, j, t):
    pre_item = get_session(u)[t - 1]
    return U[u] @ V[j].T + P[pre_item] @ Q[j].T

In [5]:
# 训练模型
for epoch in tqdm(range(epochs)):
    train_dataloader = DataLoader(train_dataset, shuffle=True)
    for user, item, index, timestamp in train_dataloader:
        if index == 0:
            continue
        # 计算第一个值
        r1 = predict_rule(user, item, index)
        # 计算第二个值
        j = random.choice(I_not_u(user))
        r2 = predict_rule(user, j, index)
        # 计算梯度
        pre_item = get_session(user)[index - 1]
        e = sigmoid(r2 - r1)
        delta_U = alpha_u * U[user] - e * (V[item] - V[j])
        delta_Vi = alpha_v * V[item] - e * U[user]
        delta_Vj = alpha_v * V[j] + e * U[user]
        delta_Qi = alpha_q * Q[item] - e * P[pre_item]
        delta_Qj = alpha_q * Q[j] + e * P[pre_item]
        delta_Ppre = alpha_p * P[pre_item] - e * (Q[item] - Q[j])
        # 更新参数
        U[user] -= lr * delta_U
        V[item] -= lr * delta_Vi
        V[j] -= lr * delta_Vj
        Q[item] -= lr * delta_Qi
        Q[j] -= lr * delta_Qj
        P[pre_item] -= lr * delta_Ppre

100%|██████████| 100/100 [17:50<00:00, 10.70s/it]


In [8]:
def FPMC(u, j):
    label = get_label(u)
    return U[u] @ V[j].T + P[label] @ Q[j].T

In [ ]:
predict(FPMC, k)

Pre: 0.0198
Rec: 0.3955
